In [1]:
!pip install cohere langchain faiss-cpu sentence-transformers pdfplumber fpdf

In [15]:
!pip install langchain-cohere

In [1]:
#Import des bibliothèques
import pdfplumber
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain.llms import Cohere
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [2]:
#Chargement du CV
import pdfplumber

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

file_path = "C:/Users/maserati/Desktop/PROJET LLM IA/CV LLM 2.pdf"
document_text = extract_text_from_pdf(file_path)

print("Contenu extrait du PDF :\n", document_text)


Contenu extrait du PDF :
 HACHMI ILHAM
ÉTUDIANTE EN M1 ECONOMETRIE - STATISTIQUES & DU INTELLIGENCE
ARTIFICIELLE
CONTACT
07.82.76.15.45
ilham.hmi@outlook.com
LOGICIELS
Pack Office, R / RStudio, LyX / LaTeX, Jabref, Python, SAS
LANGUES
Français (maternelle), Anglais (C1)
CENTRES D’INTÉRÊT
Lecture, Peinture, Volleyball Natation, Jeux de réflexion
FORMATIONS
Double diplôme - Master MASERATI Data Science & DU 2024 - Aujourd’hui
Graduate Program Intelligence Artificielle
Université Paris Est Créteil
Maitrise des langages R, Python et SAS pour le traitement et la valorisation de
l’information
Licence Économie - Gestion 2024
Université d’Evry Paris-Saclay
Majeure économie
Validée avec mention TB
Choix d’options de Science des données, Mathématiques, et Regulation & Competition Policy (en
anglais).
PROJETS
Evaluation des écarts de salaires et de leur cause en fonction du sexe (projet empirique sur données
de l’EEC).
Evaluation des écarts de salaires touchant les enfants issus de l’immigration 

In [5]:
from langchain_cohere.embeddings import CohereEmbeddings
from langchain.vectorstores import FAISS

cohere_api_key = "mmFgjgh1Sgk5pUhAQf8klaumnuHjDoZt2obCNi47"

#Embedding cohere
embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, model="embed-english-light-v2.0")

#Index faiss
vectorstore = FAISS.from_texts([document_text], embeddings)

vectorstore.save_local("faiss_index")
print("Succès")


Succès


In [16]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

#Modèle cohere
llm = Cohere(
    cohere_api_key=cohere_api_key,
    model="command-xlarge-nightly",  # Modèle optimisé pour suivre des instructions
    temperature=0.9,
    max_tokens=3000,
    
)

print("Succès")

#Template
prompt_template = """
En utilisant les informations suivantes :
{context}
Crée un CV structuré pour un poste de {role}. Incluez les sections suivantes :
1. Compétences techniques et personnelles
2. Expériences professionnelles
3. Formation académique
4. Contact

Rendez le texte clair et formel.
"""
prompt = PromptTemplate(input_variables=["context", "role"], template=prompt_template)

print("Succès")


Succès
Succès


In [17]:
#Pipeline
from langchain.chains import RetrievalQA

#Retriever
retriever = vectorstore.as_retriever()

#Chaine RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,  # Inclure les documents sources pour traçabilité
)

print("Succès")

Succès


In [23]:
#Requête CV avec prompt simple et temperature à 0,7

role ='Data Scientist'

query = f"""
Tu es un expert en rédaction de CV professionnel.

Utilise les informations suivantes pour rédiger un CV structuré, clair et professionnel pour un poste de {role} :
{document_text}

"""

#Génération
result = qa_chain.invoke({"query": query, "role": role})

#Sortie en markdown
generated_cv_text = result["result"]
generated_cv_markdown = f"""
# Curriculum Vitae

{generated_cv_text}
"""

# Afficher le CV avec une mise en page Markdown
display(Markdown(generated_cv_markdown))


# Curriculum Vitae

Voici un exemple de CV structuré, clair et professionnel pour un poste de Data Scientist : 

# HACHMI ILHAM

## ÉTUDIANTE EN M1 ECONOMETRIE - STATISTIQUES & DU INTELLIGENCE ARTIFICIELLE

### CONTACT
- Téléphone : 07.82.76.15.45
- Email : ilham.hmi@outlook.com

### LOGICIELS
- Pack Office, R / RStudio, LyX / LaTeX, Jabref, Python, SAS

### LANGUES
- Français (maternelle)
- Anglais (C1)

### CENTRES D’INTÉRÊT
Lecture, Peinture, Volleyball, Natation, Jeux de réflexion

### FORMATIONS
- Double diplôme - Master MASERATI Data Science & DU, Université Paris Est Créteil (2024 - Aujourd’hui)
    - Graduate Program Intelligence Artificielle
- Licence Économie - Gestion, Université d’Evry Paris-Saclay (2024)
    - Majeure économie, Validée avec mention TB
    - Choix d’options : Science des données, Mathématiques, et Regulation & Competition Policy (en anglais).

### PROJETS
- Evaluation des écarts de salaires et de leur cause en fonction du sexe (projet empirique sur données de l’EEC).
- Evaluation des écarts de salaires touchant les enfants issus de l’immigration ayant été scolarisés en France
- Classification de pièces d’échec par Deep Learning en utilisant un Mask RCNN

### EXPERIENCES
- Monitrice étudiante - Bibliothèque Universitaire, UEVE puis UPEC (Sept 2023 - Aujourd’hui)
    - Accueil des usagers, Rangement, Reclassement, Prêts-Retours
- Bureau des inscriptions - Pôle Web, Université d’Evry Paris-Saclay (Juin 2024 - Aout 2024)
    - Vérification des PJ, édition des cartes étudiantes
- Professeure particulière, Superprof (Oct 2022 - Juin 2023)
    - Dispense de cours particuliers de mathématiques, d’économie et de français à des lycéens.


In [22]:
#Requête avec prompt plus complet, température à 0,7

from IPython.display import display, Markdown

role ='Data Scientist'

query = f"""
Tu es un expert en rédaction de CV professionnel.

Utilise les informations suivantes pour rédiger un CV structuré, clair et professionnel :
{document_text}

Le CV doit être conçu pour un poste de {role} et doit inclure les sections suivantes avec un format bien structuré :
0. **Nom de la candidate**
1. **Titre du CV** : Indique le poste recherché.
2. **Description du candidat**
3. **Contact** : Inclure un email et un numéro de téléphone.
4. **Compétences techniques et personnelles** : Liste clairement les compétences pertinente (n'incluant pas les centres d'intérêts).
5. **Formation** : Présente les diplômes obtenus avec l'année d'obtention et les établissements, et fait un descriptif des compétences acquises par ces diplômes.
6. **Projets**
7. **Expériences professionnelles** : Décris les expériences les plus significatives en indiquant les responsabilités, les résultats obtenus, et les dates correspondantes.


Rends le texte clair, formel, et impactant. Utilise un ton professionnel et inspire confiance au lecteur.

"""


#Génération
result = qa_chain.invoke({"query": query, "role": role})

#Sortie en markdown
generated_cv_text = result["result"]
generated_cv_markdown = f"""
# Curriculum Vitae

{generated_cv_text}
"""

#Sortie
display(Markdown(generated_cv_markdown))


# Curriculum Vitae

# HACHMI ILHAM - Data Scientist

## Candidate Description: 

Étudiante sérieuse et motivée en M1 Econométrie - Statistiques & DU Intelligence Artificielle, je suis à la recherche d'un poste de Data Scientist pour mettre en pratique mes compétences techniques et mon intérêt pour l'analyse de données. 

## Contact:
- Téléphone: 07.82.76.15.45
- Email: ilham.hmi@outlook.com

## Compétences techniques et personnelles:
- Logiciels: Pack Office, R / RStudio, LyX / LaTeX, Jabref, Python, SAS
- Langues: Français (maternelle), Anglais (niveau C1)
- Forte capacité d'analyse et de traitement de données.
- Maîtrise des méthodes statistiques et économétriques avancées.
- Expérience dans la programmation et l'utilisation d'outils d'intelligence artificielle.

## Formation:
- **Double diplôme - Master MASERATI Data Science & DU** (2024 - Aujourd’hui), Université Paris Est Créteil:
  - Apprentissage approfondi des langages de programmation R, Python et SAS pour le traitement de données.
  - Développement de compétences en analyse et valorisation de l'information.

- **Licence Économie - Gestion** (2024), Université d’Evry Paris-Saclay:
  - Majeure économie obtenue avec mention Très Bien.
  - Options choisies : Science des données, Mathématiques, et Regulation & Competition Policy (en anglais). Ces options ont renforcé mes compétences en matière de modélisation et d'analyse économique.

## Projets:
- Evaluation des écarts de salaires et de leur cause en fonction du sexe, sur des données de l’EEC.
- Etude des disparités salariales affectant les enfants d'immigrés scolarisés en France.
- Classification de pièces d’échec par Deep Learning en utilisant un Mask RCNN.

## Expériences professionnelles:
- **Monitrice étudiante - Bibliothèque Universitaire** (Sept 2023 - Aujourd’hui), UEVE puis UPEC:
  - Accueil et assistance des usagers, gestion du rangement et reclassement, prêts et retours de documents.

- **Bureau des inscriptions - Pôle Web**, Université d’Evry Paris-Saclay:
  - Vérification des pièces jointes et édition des cartes étudiantes (Juin-Août 2024).

- **Professeure particulière**, Superprof (Oct 2022 - Juin 2023):
  - Cours particuliers de mathématiques, d’économie et de français pour des lycéens. Ce poste m'a permis de développer mes compétences pédagogiques et ma capacité à transmettre des connaissances complexes de manière claire et concise.


In [12]:
from IPython.display import display, Markdown

In [13]:
#Requête avec prompt complet et température à 0,3

role ='Data Scientist'

query = f"""
Tu es un expert en rédaction de CV professionnel.

Utilise les informations suivantes pour rédiger un CV structuré, clair et professionnel :
{document_text}

Le CV doit être conçu pour un poste de {role} et doit inclure les sections suivantes avec un format bien structuré :
0. **Nom de la candidate**
1. **Titre du CV** : Indique le poste recherché.
2. **Description du candidat**
3. **Contact** : Inclure un email et un numéro de téléphone.
4. **Compétences techniques et personnelles** : Liste clairement les compétences pertinente (n'incluant pas les centres d'intérêts).
5. **Formation** : Présente les diplômes obtenus avec l'année d'obtention et les établissements, et fait un descriptif des compétences acquises par ces diplômes.
6. **Projets**
7. **Expériences professionnelles** : Décris les expériences les plus significatives en indiquant les responsabilités, les résultats obtenus, et les dates correspondantes.


Rends le texte clair, formel, et impactant. Utilise un ton professionnel et inspire confiance au lecteur.

"""

#Génération
result = qa_chain.invoke({"query": query, "role": role})

#Sortie en markdown
generated_cv_text = result["result"]
generated_cv_markdown = f"""
# Curriculum Vitae

{generated_cv_text}
"""

#Sortie
display(Markdown(generated_cv_markdown))


# Curriculum Vitae

# Hachmi Ilham - Data Scientist

## Description
Étudiante en M1 Econométrie - Statistiques & DU Intelligence Artificielle, je suis passionnée par l'analyse de données et l'utilisation de l'IA pour résoudre des problèmes complexes. Mon objectif est de mettre mes compétences au service d'une entreprise innovante et de contribuer à l'amélioration de ses processus décisionnels.

## Contact
- Email: ilham.hmi@outlook.com
- Téléphone: 07.82.76.15.45

## Compétences techniques et personnelles
- Maîtrise des langages de programmation: R, Python, SAS
- Connaissances avancées en statistiques et économétrie
- Expérience avec les outils de visualisation de données (RStudio, LyX/LaTeX)
- Gestion de bases de données avec Jabref
- Niveau C1 en anglais
- Capacité à travailler en équipe et à communiquer efficacement
- Sens de l'organisation et de la gestion du temps

## Formation
- **Double diplôme - Master MASERATI Data Science & DU** (2024 - Aujourd'hui), Université Paris Est Créteil
  - Développement de compétences avancées en Data Science, notamment dans la maîtrise des langages R, Python et SAS pour le traitement et l'analyse de données.

- **Licence Économie - Gestion** (2024), Université d’Evry Paris-Saclay
  - Majeure en économie avec mention TB.
  - Options choisies: Science des données, Mathématiques, et Regulation & Competition Policy (en anglais), me permettant d'acquérir des connaissances approfondies en analyse de données et en modélisation économique.

## Projets
- **Evaluation des écarts de salaires** : Analyse empirique des données de l’EEC pour étudier les disparités salariales entre les sexes et identifier les causes sous-jacentes.
- **Impact de l'immigration sur les salaires** : Étude des écarts de salaires chez les enfants issus de l'immigration scolarisés en France.
- **Classification de pièces d’échec par Deep Learning** : Utilisation d'un Mask RCNN pour la classification et la reconnaissance d'images de pièces d'échec.

## Expériences professionnelles
- **Monitrice étudiante - Bibliothèque Universitaire** (Sept 2023 - Aujourd’hui), UEVE puis UPEC
  - Accueil et assistance des usagers, gestion du rangement et du reclassement des ouvrages, gestion des prêts et retours de documents.

- **Bureau des inscriptions - Pôle Web** (Juin 2024 - Aout 2024), Université d’Evry Paris-Saclay
  - Vérification des pièces jointes et édition des cartes étudiantes, assurant la précision et la confidentialité des informations.

- **Professeure particulière** (Oct 2022 - Juin 2023), Superprof
  - Dispensé des cours particuliers en mathématiques, économie et français à des lycéens, adaptant mes méthodes d'enseignement aux besoins individuels.

Ce CV met en avant les compétences et expériences de Hachmi Ilham, démontrant sa capacité à analyser et interpréter des données complexes, ainsi que son expertise dans le domaine de l'intelligence artificielle. Son parcours académique et ses projets démontrent sa passion pour la Data Science et sa motivation à contribuer à des projets innovants.


In [21]:
#Requête avec prompt complet et température à 0,9

role ='Data Scientist'

query = f"""
Tu es un expert en rédaction de CV professionnel.

Utilise les informations suivantes pour rédiger un CV structuré, clair et professionnel :
{document_text}

Le CV doit être conçu pour un poste de {role} et doit inclure les sections suivantes avec un format bien structuré :
0. **Nom de la candidate**
1. **Titre du CV** : Indique le poste recherché.
2. **Description du candidat**
3. **Contact** : Inclure un email et un numéro de téléphone.
4. **Compétences techniques et personnelles** : Liste clairement les compétences pertinente.
5. **Formation** : Présente les diplômes obtenus avec l'année d'obtention et les établissements, et fait un descriptif des compétences acquises par ces diplômes.
6. **Projets**
7. **Expériences professionnelles** : Décris les expériences les plus significatives en indiquant les responsabilités, les résultats obtenus, et les dates correspondantes.


Rends le texte clair, formel, et impactant. Utilise un ton professionnel et inspire confiance au lecteur.

"""

#Génération
result = qa_chain.invoke({"query": query, "role": role})

#Sortie en markdown
generated_cv_text = result["result"]
generated_cv_markdown = f"""
# Curriculum Vitae

{generated_cv_text}
"""

#Sortie
display(Markdown(generated_cv_markdown))


# Curriculum Vitae

# CV de HACHMI ILHAM

## Data Scientist

Je suis actuellement étudiante en M1 Econométrie, Statistiques et Intelligence Artificielle, avec une forte passion pour l'analyse de données et la modélisation statistique. J'ai acquis une solide expérience en traitement de données et en intelligence artificielle au cours de mon parcours académique et professionnel.

### Contact
- Email : ilham.hmi@outlook.com
- Téléphone : 07.82.76.15.45

### Compétences techniques et personnelles
- **Langues** : Français (maternel), Anglais (C1)
- **Informatique** : Pack Office, R / RStudio, LyX / LaTeX, Jabref, Python, SAS, Deep Learning (Mask RCNN)

Mes compétences techniques comprennent la maîtrise d'outils de bureautique et de logiciels spécialisés pour l'analyse et la visualisation de données, ainsi que la programmation en Python et en SAS. J'ai également une expérience en Deep Learning pour la classification d'images.

Au niveau personnel, je suis une personne curieuse, organisée et axée sur les résultats. Je suis capable de travailler de manière indépendante et de collaborer efficacement au sein d'une équipe.

### Formation
- **Double diplôme - Master MASERATI Data Science & DU** (2024 - Aujourd'hui)
  - Université Paris Est Créteil
  - J'acquière une expertise dans l'application de l'intelligence artificielle au traitement des données, avec un focus sur les langages R, Python et SAS.

- **Licence Économie - Gestion** (2024), Université d’Evry Paris-Saclay
  - *Majeure économie - Mention TB*
  - Options choisies : Science des données, Mathématiques, et Regulation & Competition Policy (en anglais).

Au cours de ma formation, j'ai développé des compétences solides en économétrie et en statistiques, ainsi qu'en science des données. J'ai appris à manipuler et analyser de larges jeux de données, et à construire des modèles prédictifs et explicatifs. 

### Projets
- *Evaluation des écarts de salaires*: études empiriques sur les différences de salaires en fonction du sexe et de l'origine.
- *Classification de pièces d’échec par Deep Learning*: utilisation d'un Mask RCNN pour détecter et classifier des pièces d'échec dans des images.

J'ai appliqué mes compétences analytiques et techniques pour mener à bien des projets de recherche en économétrie et en apprentissage automatique, avec un focus sur l'éthique et la diversité. 

### Expériences professionnelles
- **Monitrice étudiante**, Bibliothèque Universitaire (Sept 2023 - Aujourd’hui)
  - UEVE puis UPEC
  - Responsable de l'accueil et du service aux usagers, du rangement et du prêt/retour de documents.

- **Bureau des inscriptions**, Université d’Evry Paris-Saclay (Juin 2024 - Aout 2024)
  - Vérification des documents et édition des cartes étudiantes.

- **Professeure particulière**, Superprof (Oct 2022 - Juin 2023)
  - Cours particuliers de mathématiques, d’économie et de français à des lycéens, adaptés à leurs besoins et niveaux.

J'ai su démontrer mes aptitudes en communication, organisation et pédagogie à travers mes expériences professionnelles, ainsi que ma capacité à travailler avec différents types de publics.
